# Dynamic Configuration and Chaining in Mirascope

This notebook provides a detailed introduction to using Dynamic Configuration and Chaining in Mirascope. We'll cover various examples ranging from basic usage to more complex chaining techniques.

For more information on these topics, refer to the following documentation:
- [Dynamic Configuration](https://docs.mirascope.io/learn/dynamic_configuration/)
- [Chaining](https://docs.mirascope.io/learn/chaining/)

1. [Setup](#Setup)
2. [Dynamic Configuration](#Dynamic-Configuration)
   - [Basic Usage](#Basic-Usage)
   - [Computed Fields](#Computed-Fields)
   - [Dynamic Tools](#Dynamic-Tools)
3. [Chaining](#Chaining)
   - [Basic Chaining](#Basic-Chaining)
   - [Chaining with Computed Fields](#Chaining-with-Computed-Fields)
   - [Multi-Step Chaining](#Multi-Step-Chaining)
   - [Conditional Chaining](#Conditional-Chaining)
   - [Parallel Chaining](#Parallel-Chaining)
4. [Advanced Techniques](#Advanced-Techniques)
   - [Combining Dynamic Config and Chaining](#Combining-Dynamic-Config-and-Chaining)
   - [Error Handling in Chains](#Error-Handling-in-Chains)
5. [Conclusion](#Conclusion)

## Setup

First, let's install Mirascope and set up our environment. We'll use OpenAI for our examples, but you can adapt these to other providers supported by Mirascope. For more information on supported providers, see the [Calls documentation](https://docs.mirascope.io/learn/calls/).

In [ ]:
!pip install "mirascope[openai]"

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

## Dynamic Configuration

Dynamic Configuration in Mirascope allows you to modify the behavior of LLM calls at runtime based on input arguments or other conditions. For more details, see the [Dynamic Configuration documentation](https://docs.mirascope.io/learn/dynamic_configuration/).

### Basic Usage

In [1]:
from mirascope.core import openai, prompt_template


@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book")
def recommend_book(genre: str, creativity: float) -> openai.OpenAIDynamicConfig:
    return {"call_params": {"temperature": creativity}}


# Low creativity recommendation
response = recommend_book("mystery", 0.2)
print("Low creativity:", response.content)

# High creativity recommendation
response = recommend_book("mystery", 0.8)
print("High creativity:", response.content)

Low creativity: I recommend "The Girl with the Dragon Tattoo" by Stieg Larsson. This gripping mystery novel follows journalist Mikael Blomkvist and hacker Lisbeth Salander as they investigate the decades-old disappearance of a wealthy industrialist's niece. The story is filled with twists, complex characters, and a deep exploration of themes such as corruption and family secrets. It's the first book in the Millennium series and has captivated readers with its intricate plot and engaging writing. Enjoy the read!
High creativity: I recommend "The Girl with the Dragon Tattoo" by Stieg Larsson. This gripping mystery follows journalist Mikael Blomkvist and hacker Lisbeth Salander as they investigate the decades-old disappearance of a young woman from a powerful family. With its complex characters, intricate plot, and dark themes, it’s a compelling read that keeps you guessing until the very end. Enjoy!


### Computed Fields

Computed fields allow you to dynamically generate or modify template variables used in your prompt. For more information on prompt templates, see the [Prompts documentation](https://docs.mirascope.io/learn/prompts/).

In [1]:
@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book with a reading level of {reading_level}")
def recommend_book_by_age(genre: str, age: int) -> openai.OpenAIDynamicConfig:
    reading_level = "adult"
    if age < 12:
        reading_level = "elementary"
    elif age < 18:
        reading_level = "young adult"
    return {"computed_fields": {"reading_level": reading_level}}


response = recommend_book_by_age("fantasy", 15)
print(response.content)

I recommend **"The Cruel Prince" by Holly Black**. It's a captivating young adult fantasy that follows Jude, a mortal girl living in the treacherous world of Faerie. The story is filled with political intrigue, betrayal, and magic, as Jude navigates her place among the powerful fae, all while trying to prove herself worthy of a role in their world. The book is the first in "The Folk of the Air" trilogy, offering an engaging blend of adventure and romance that appeals to young adult readers.


### Dynamic Tools

You can dynamically configure which tools are available to the LLM based on runtime conditions.

In [1]:
from mirascope.core import BaseToolKit, toolkit_tool


class BookToolkit(BaseToolKit):
    genre: str

    @toolkit_tool
    def format_book(self, title: str, author: str) -> str:
        """Format a {self.genre} book recommendation."""
        return f"{title} by {author} ({self.genre})"


@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book")
def recommend_book_with_tool(genre: str) -> openai.OpenAIDynamicConfig:
    toolkit = BookToolkit(genre=genre)
    return {"tools": toolkit.create_tools()}


response = recommend_book_with_tool("mystery")
if response.tool:
    print(response.tool.call())
else:
    print(response.content)

The Girl with the Dragon Tattoo by Stieg Larsson (mystery)


## Chaining

Chaining in Mirascope allows you to combine multiple LLM calls or operations in a sequence to solve complex tasks.

### Basic Chaining

In [1]:
@openai.call("gpt-4o-mini")
@prompt_template("Summarize this text: {text}")
def summarize(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {text}")
def translate(text: str, language: str): ...


original_text = "Long English text here..."
summary = summarize(original_text)
translation = translate(summary.content, "french")
print(translation.content)

Veuillez fournir le texte que vous souhaitez résumer, et je serai heureux de vous aider !


### Chaining with Computed Fields

Using computed fields for chaining provides better traceability and debugging capabilities.

In [1]:
@openai.call("gpt-4o-mini")
@prompt_template("Summarize this text: {text}")
def summarize(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {summary}")
def summarize_and_translate(text: str, language: str) -> openai.OpenAIDynamicConfig:
    return {"computed_fields": {"summary": summarize(text)}}


response = summarize_and_translate("Long English text here...", "french")
print(response.content)
print("\nComputed fields:", response.dynamic_config["computed_fields"])

Bien sûr ! Veuillez fournir le texte que vous souhaitez que je résume.

Computed fields: {'summary': OpenAICallResponse(metadata={}, response=ChatCompletion(id='chatcmpl-A5n1iB4auyKOofKmGbUJ3Y6y78Ivx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure! Please provide the text you'd like me to summarize.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725943462, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_483d39d857', usage=CompletionUsage(completion_tokens=12, prompt_tokens=18, total_tokens=30)), tool_types=None, prompt_template='Summarize this text: {text}', fn_args={'text': 'Long English text here...'}, dynamic_config=None, messages=[{'role': 'user', 'content': 'Summarize this text: Long English text here...'}], call_params={}, call_kwargs={'model': 'gpt-4o-mini', 'messages': [{'role': 'user', 'content': 'Summarize this text: Long English 

### Multi-Step Chaining

Let's create a more complex chain that involves multiple steps.

In [1]:
@openai.call("gpt-4o-mini")
@prompt_template("Generate a short story in the {genre} genre")
def generate_story(genre: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Summarize this story in one sentence: {story}")
def summarize_story(story: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {text}")
def translate(text: str, language: str): ...


@openai.call("gpt-4o-mini")
@prompt_template(
    """
    Original Story: {original_story}
    Summary: {summary}
    Translated Summary: {translated_summary}
    
    Based on the above information, analyze the effectiveness of the summary and translation.
    """
)
def analyze_chain(
    original_story: str, summary: str, translated_summary: str
) -> openai.OpenAIDynamicConfig:
    return {
        "computed_fields": {
            "original_story": original_story,
            "summary": summary,
            "translated_summary": translated_summary,
        }
    }


# Execute the chain
story = generate_story("science fiction").content
summary = summarize_story(story).content
translated_summary = translate(summary, "spanish").content
analysis = analyze_chain(story, summary, translated_summary)

print("Analysis:", analysis.content)

Analysis: The summary and translation provided are effective for several reasons:

### Summary Analysis:
1. **Conciseness**: The summary captures the essential elements of the original story without extraneous details. It succinctly conveys the main plot points, including the discovery of the ancient space station, the encounter with the Xylthari civilization, and the resolution of the conflict.

2. **Clarity**: The language is clear and straightforward, making it easy for readers to understand the core narrative. Key characters, actions, and themes are presented in a way that maintains the story's integrity.

3. **Character and Theme**: The summary effectively conveys the motivations of Captain Elara Quinn and the significance of the crew's decision to free the trapped phantoms, emphasizing themes of empathy, sacrifice, and exploration.

### Translation Analysis:
1. **Accurate Vocabulary**: The translation accurately reflects the original summary's vocabulary and tone, using appropria

### Conditional Chaining

We can use conditional logic to determine the next step in our chain.

In [1]:
from pydantic import BaseModel


class SentimentResponse(BaseModel):
    sentiment: str


@openai.call("gpt-4o-mini", response_model=SentimentResponse)
@prompt_template("Analyze the sentiment of this text: {text}")
def analyze_sentiment(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Rewrite this text to be more positive: {text}")
def make_positive(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Summarize this text: {text}")
def summarize(text: str): ...


def process_text(text: str):
    sentiment = analyze_sentiment(text).sentiment
    if sentiment == "negative":
        return make_positive(text).content
    else:
        return summarize(text).content


negative_text = "I had a terrible day. Everything went wrong."
positive_text = "Today was amazing! I accomplished all my goals."

print("Processed negative text:", process_text(negative_text))
print("\nProcessed positive text:", process_text(positive_text))

Processed negative text: Today presented some challenges, but I'm looking forward to tomorrow and the opportunities it will bring!

Processed positive text: Today was great! I achieved all my goals.


### Parallel Chaining

We can execute multiple chains in parallel for improved efficiency.

In [1]:
import asyncio


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {text}")
async def translate_async(text: str, language: str): ...


async def translate_to_multiple_languages(text: str, languages: list[str]):
    tasks = [translate_async(text, lang) for lang in languages]
    results = await asyncio.gather(*tasks)
    return {
        lang: result.content for lang, result in zip(languages, results, strict=False)
    }


text = "Hello, world!"
languages = ["French", "Spanish", "German", "Italian"]


async def main():
    translations = await translate_to_multiple_languages(text, languages)
    for lang, translation in translations.items():
        print(f"{lang}: {translation}")


# Run the asynchronous function in Jupyter notebooks
await main()

# Run the asynchronous function in a Python script
# asyncio.run(main())

French: Bonjour, le monde !
Spanish: ¡Hola, mundo!
German: Hallo, Welt!
Italian: Ciao, mondo!


## Advanced Techniques

### Combining Dynamic Config and Chaining

We can combine dynamic configuration with chaining for more complex workflows.

In [1]:
@openai.call("gpt-4o-mini")
@prompt_template("Generate a {length} {genre} story")
def generate_story(genre: str, length: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Analyze the tone of this story: {story}")
def analyze_tone(story: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Rewrite this {genre} story in a {tone} tone: {story}")
def rewrite_story(story: str, genre: str, tone: str) -> openai.OpenAIDynamicConfig:
    return {
        "computed_fields": {
            "story": story,
            "genre": genre,
            "tone": analyze_tone(story).content,
        }
    }


original_story = generate_story("mystery", "short").content
rewritten_story = rewrite_story(original_story, "mystery", "suspenseful")

print("Original Story:", original_story)
print("\nRewritten Story:", rewritten_story.content)
print("\nComputed Fields:", rewritten_story.dynamic_config["computed_fields"])

Original Story: **Title: The Whispering Willows**

The small town of Eldermere was known for its scenic beauty, but beneath its charming façade lay secrets buried deep as the roots of its ancient weeping willows. One brisk autumn evening, Clara, an aspiring journalist, returned to her childhood home, drawn by a strange feeling that had tugged at her heart since her grandmother's passing.

As she sorted through her grandmother's belongings, Clara stumbled upon an old wooden box, intricately carved with patterns of intertwining vines and leaves. Inside, she found a collection of yellowed letters and a delicate silver locket. Intrigued, she opened the locket, revealing two portraits: one of her grandmother as a young woman and another of an unfamiliar man with dark, piercing eyes.

Clara’s curiosity was piqued. She had heard whispers about her grandmother’s past—rumors of a forbidden love, a tragedy that had marred their family’s history. Determined to uncover the truth, Clara visited the

### Error Handling in Chains

Implementing robust error handling is crucial in complex chains.

In [1]:
from openai import OpenAIError


@openai.call("gpt-4o-mini")
@prompt_template("Summarize this text: {text}")
def summarize(text: str): ...


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {text}")
def translate(text: str, language: str): ...


def process_text_with_error_handling(text: str, target_language: str):
    try:
        summary = summarize(text).content
    except OpenAIError as e:
        print(f"Error during summarization: {e}")
        summary = text  # Fallback to original text if summarization fails

    try:
        translation = translate(summary, target_language).content
        return translation
    except OpenAIError as e:
        print(f"Error during translation: {e}")
        return summary  # Fallback to summary if translation fails


result = process_text_with_error_handling("Long text here...", "French")
print("Processed Result:", result)

Processed Result: Bien sûr ! Veuillez fournir le texte que vous souhaitez que je résume, et je serai heureux de vous aider !


## Conclusion

This notebook has demonstrated various techniques for using Dynamic Configuration and Chaining in Mirascope. These powerful features allow you to create flexible, efficient, and complex LLM-powered applications. By combining these techniques, you can build sophisticated AI systems that can adapt to different inputs and requirements while maintaining robustness and traceability.

Remember to always consider error handling, especially in complex chains, to ensure your applications are resilient to potential issues that may arise during LLM calls or processing steps.